In [187]:
import pandas as pd
import matplotlib.pyplot as plt

# Load Sample Data

In [188]:
df = pd.read_excel('./files/shoe_data.xlsx', skiprows=[0,1,2], sheet_name=0)
del df[df.columns[-1]]
del df[df.columns[-1]]
del df[df.columns[-1]]
del df[df.columns[0]]
df.tail(2)

,InvoiceNo,Date,Country,ProductID,Shop,Gender,Size (US),Size (Europe),Size (UK),UnitPrice,Discount,Unnamed: 12,Month,SalePrice
14965,65776,2016-12-31 17:54:24.000018,Germany,2231,GER1,Female,9.5,40,7.5,199,0.0,2016,12,199.0
14966,65777,2016-12-31 18:52:00.000002,Germany,2156,GER1,Female,6.5,37,4.5,139,0.1,2016,12,125.1


## Segment data by Gender


In [189]:
df_men = df[df.Gender == 'Male']
df_men.reset_index(inplace=True)
df_men.drop('index', axis=1, inplace=True)

df_female = df[df.Gender == 'Female']
df_female.reset_index(inplace=True)
df_female.drop('index', axis=1, inplace=True)

/Users/natitaw/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Frequency Distributions
- Shoe Size

In [190]:
df_men.groupby(['Size (US)', 'Country']).count().columns

Index(['InvoiceNo', 'Date', 'ProductID', 'Shop', 'Gender', 'Size (Europe)',
       'Size (UK)', 'UnitPrice', 'Discount', 'Unnamed: 12', 'Month',
       'SalePrice'],
      dtype='object')

In [191]:
df_men.groupby(['Size (US)', 'Country']).count().unstack(level=1).InvoiceNo

Country,Canada,Germany,United Kingdom,United States
Size (US),,,,
6.0,15,30,6,54
6.5,15,18,12,45
7.0,24,30,21,39
7.5,45,48,12,66
8.0,51,117,45,141
8.5,192,174,87,225
9.0,324,348,183,492
9.5,375,549,225,741
10.0,237,411,156,543


## What is the number of shoes that are likely to be sold, based on historical data?

- Plan - Find the 95% confidence interval using
    - Take last 12 months of sales 
    - Only for men shoes as the problem is identical for women
    - Only for the USA

In [148]:
df_men_usa = df_men[df_men.Country == 'United States']
df_men_usa.reset_index(inplace=True)

In [149]:
df_men_usa = df_men_usa.groupby(['Size (US)', 'Month']).count().unstack(level=1).InvoiceNo.fillna(0)

In [153]:
mean = pd.DataFrame(df_men_usa).mean(axis=1)
std_ = pd.DataFrame(df_men_usa).std(axis=1)

In [167]:
import math, numpy as np
from scipy import stats
interval = []

td = stats.t(12-1)


print (stats.t.ppf(1-0.025, 12))

for m, s in zip(mean, std_):
    se = s/math.sqrt(12)
    
    ci = m + np.array([-1, 1])*td.isf(0.025)*se
    
    interval.append(ci)

2.1788128296634177


In [184]:
df_men_usa['interval'] = interval
df_men_usa['interval'] = df_men_usa['interval'].apply(lambda xy: (round(xy[0]), round(xy[1])))

In [185]:
df_men_usa

Month,1,2,3,4,5,6,7,8,9,10,11,12,interval
Size (US),,,,,,,,,,,,,
6.0,4.0,3.0,3.0,1.0,6.0,4.0,4.0,7.0,8.0,11.0,3.0,0.0,"(3.0, 6.0)"
6.5,7.0,3.0,0.0,4.0,0.0,0.0,3.0,10.0,5.0,2.0,5.0,6.0,"(2.0, 6.0)"
7.0,0.0,1.0,2.0,0.0,9.0,6.0,5.0,2.0,4.0,4.0,3.0,3.0,"(2.0, 5.0)"
7.5,3.0,3.0,7.0,2.0,9.0,0.0,10.0,5.0,8.0,9.0,7.0,3.0,"(3.0, 8.0)"
8.0,14.0,12.0,12.0,5.0,21.0,8.0,17.0,5.0,16.0,12.0,7.0,12.0,"(9.0, 15.0)"
8.5,18.0,17.0,14.0,14.0,20.0,24.0,23.0,25.0,16.0,25.0,15.0,14.0,"(16.0, 22.0)"
9.0,32.0,33.0,30.0,30.0,53.0,44.0,54.0,52.0,51.0,49.0,31.0,33.0,"(34.0, 48.0)"
9.5,44.0,59.0,69.0,48.0,52.0,63.0,60.0,77.0,64.0,69.0,77.0,59.0,"(55.0, 68.0)"
10.0,33.0,45.0,44.0,34.0,26.0,62.0,56.0,53.0,53.0,58.0,36.0,43.0,"(38.0, 52.0)"
